In [1]:
def coefs_1d(N,N0,lab) :
    return vector([ var(lab+'%s'%i) for i in range(N0,N0+N) ])


#Generates a 1-D polynomial as a sage expreession based on a set of coefficients
def poly_1d(N,coefs,x) :
    return sum( vector([ coefs[i]*x^i for i in range(N) ]) )


def points_gll_to_coefs(N,dx) :
    var('x')
    coefs = coefs_1d(N,0,'a')
    p = poly_1d(N,coefs,x)
    gll = points_gll(N)
    pnts = vector([ p.subs(x=gll[i]*dx) for i in range(N) ])
    coefs_to_pnts = jacobian(pnts,coefs)
    pnts_to_coefs = coefs_to_pnts^-1
    return pnts_to_coefs,coefs_to_pnts


def stencil_to_coefs(N,dx) :
    var('x')
    hs = (N-1)/2
    coefs = coefs_1d(N,0,'a')
    p = poly_1d(N,coefs,x)
    constr = vector([ integrate(p,x,(2*i-1)/2*dx,(2*i+1)/2*dx)/dx for i in range(-hs,hs+1) ])
    coefs_to_constr = jacobian(constr,coefs)
    constr_to_coefs = coefs_to_constr^-1
    return constr_to_coefs,coefs_to_constr


def coefs_to_deriv(N,dx) :
    var('x')
    coefs = coefs_1d(N,0,'a')
    p = poly_1d(N,coefs,x)
    dp = diff(p,x)
    deriv_coefs = compute_coefs(N,dp,x)
    coefs_to_deriv = jacobian(deriv_coefs,coefs)
    return coefs_to_deriv


#Computes the coefficients of an Nth-order polynomial expression and returns a sage vector
#Mostly just a helper function for integrate_poly
def compute_coefs(N,p,x) :
    x = var(str(x))
    return vector([ diff(p,x,i).subs({x:0})/factorial(i) for i in range(N) ])


############################################################################
## Computes Gauss-Lobatto quadrature weights and nodes to integrate over
## the domain [-1,1]. Adapted from matlab algorithm in
## http://www.mathworks.com/matlabcentral/fileexchange/4775-legende-gauss-lobatto-nodes-and-weights
############################################################################
## nnodes  : Total number of integration points
## prec    : Floating Point Precision of the result
## verbose : Print things as you iterations proceed? (True or False)
## tol     : Tolerance for L-infinity of difference in old and new guesses
## mxcount : Maximum number of allowed iterations
############################################################################
def lobatto_weights_nodes(nnodes,prec,verbose,tol,mxcount) :
    N1 = nnodes
    N = N1-1
    x = vector( [ cos(pi*(i)/(N)+pi) for i in range(N1) ] )
    x = x.n(prec)
    P = matrix( N1 , N1 , [ 0.n(prec) for i in range(N1^2)] )
    xold = x * 2
    linf = max(x-xold)/max(x)
    count = 0
    while ( linf > tol and count < mxcount ) :
        xold = x
        P[:,0] = 1
        P[:,1] = x
        for k in range(1,N) :
            tmp = matrix(N1,1, [ x[i] * P[i,k] for i in range(N1) ] )
            P[:,k+1] = ( (2*(k+1)-1)*tmp - ((k+1)-1)*P[:,k-1] )/(k+1)
        tmp  = matrix(N1,1, [ x[i] * P[i,N] - P[i,N-1] for i in range(N1) ] )
        tmp2 = matrix(N1,1, [ N1*P[i,N] for i in range(N1) ] )
        tmp3 = vector( [ tmp[i,0] / tmp2[i,0] for i in range(N1) ] )
        x = xold - tmp3
        linf = max(x-xold)/max(x)
        count = count + 1
        if (verbose) :
            print(count,linf)
    w = vector( [ 2 / (N*N1*P[i,N]^2) for i in range(N1) ] )
    return x,w


def tanh_cluster(N,mu) :
    return vector( [ (tanh((2*mu*i)/(N-1)-mu)/tanh(mu)+1)/2-1/2 for i in range(N) ] )


#Returns N GLL point locations in [-0.5,0.5] (quad precision)
def points_gll(N) :
    x,w = lobatto_weights_nodes(N,129,False,1e-35,20)
    return vector([ x[i]/2 for i in range(N) ])



In [6]:
N = 5
var('dx')

# Compute gll to deriv to gll
g2c,c2g = points_gll_to_coefs(N,dx)
s2c,c2s = stencil_to_coefs(N,dx)
c2d     = coefs_to_deriv(N,dx)

s_to_g = c2g * s2c
g2d2g = c2g * c2d * g2c

stenVals = vector([1.,0.,2.,0.5,3.])

# GLL-based
print("GLL-based")
print( g2d2g * g2d2g * g2d2g * s_to_g * stenVals )

# True values
coefs = s2c * stenVals
p = poly_1d(N,coefs,x)
d3p = diff(p,x,3)
coefs_d3 = compute_coefs(N,d3p,x)
print("\nExact values")
print(c2g * coefs_d3)


GLL-based
(-6.50000000000000/dx^3, -4.08257569495584/dx^3, 0.500000000000000/dx^3, 5.08257569495584/dx^3, 7.50000000000000/dx^3)

Exact values
(-6.50000000000000/dx^3, -4.08257569495584/dx^3, 0.500000000000000/dx^3, 5.08257569495584/dx^3, 7.50000000000000/dx^3)
